In [1]:
config = {
    'time': '01:00',
    
    'auto_size': False,
    'W': 660, # ignored when auto_size = True
    'H': 200, # ignored when auto_size = True
    
    'auto_center': True,
    'auto_center_each_frame': False,
    'x': 100, # ignored when auto_center = True
    'y': 10, # ignored when auto_center = True
    
    'font_size': 200,
    'font_face': '/Library/Fonts/Arial Rounded Bold.ttf',
    #/Users/paulo/Library/Fonts/Market_Deco.ttf
    #/Users/paulo/Library/Fonts/orange juice 2.0.ttf
    #/Users/paulo/Library/Fonts/gigfont.ttf
    #/Library/Fonts/Arial.ttf
    #/Library/Fonts/Arial Bold.ttf
    #/Library/Fonts/Arial Black.ttf
    #/Library/Fonts/Bodoni 72.ttc
    #/Library/Fonts/Chalkboard.ttc
    #/System/Library/Fonts/Courier.dfont
    #/Library/Fonts/DIN Alternate Bold.ttf
    'text_color': '#000000',
    'bg_color': '#ffffff',
    
    'fps': 10,
    'make_gif': 0,
    'make_zero': 1,
    'make_mp4': 1,
    'make_static': 1
}

In [2]:
# REFERENCE
# https://stackoverflow.com/questions/43060479/how-to-get-the-font-pixel-height-using-pil-imagefont
# https://pillow.readthedocs.io/en/3.1.x/reference/ImageFont.html

import string
from PIL import Image, ImageDraw, ImageFont
import re
import datetime, time
import os
import cv2

In [3]:
def generate_countdown(config):
    time_multipliers = [1, 60, 3600] # time parts will be reversed too to acomodate all lenghts of time.

    time_string = config['time']
    font_size = int(config['font_size'])
    W = int(config['W'])
    H = int(config['H'])
    text_color = config['text_color']
    bg_color = config['bg_color']
    font_face = config['font_face']
    font_face_for_filepath = re.compile('[^0-9a-z]+').sub('', os.path.basename(font_face).lower())
    fps = int(config['fps'])
    make_gif = 1 and int(config['make_gif'])
    make_zero = 1 and int(config['make_zero'])
    make_mp4 = 1 and int(config['make_mp4'])
    auto_center = 1 and int(config['auto_center'])
    auto_size = 1 and int(config['auto_size'])
    make_static = 1 and int(config['make_static'])
    auto_center_each_frame = 1 and int(config['auto_center_each_frame'])
    

    x = int(config['x'])
    y = int(config['y'])

    # PARAMETERS
    time_parts_factors = list(zip(reversed(time_string.split(':')), time_multipliers))
    time_parts_multiplied = [ int('0' + a.replace(' ','')) * b for a,b in time_parts_factors ]
    seconds = sum(time_parts_multiplied)

    if seconds > 10 * time_multipliers[2]: # 24 hours
        seconds = 10
        raise ValueError('Time can\'t be higher than 10:00:00')

    # OBJECTS and ADDITIONAL METRICS
    slice_position = 0
    if seconds < 60 * 60:
        slice_position = 2
    initial_text = str(datetime.timedelta(seconds=seconds))[slice_position:] # necessary for base calculations.
    fnt = ImageFont.truetype(font_face, font_size)
    box = fnt.getmask(initial_text).getbbox()
    if auto_size:
        W, H = int(box[2] * 1.1), int(box[3] * 1.4)
    img = Image.new('RGBA', (W,H), color = bg_color)
    draw = ImageDraw.Draw(img)
    (width, baseline), (offset_x, offset_y) = fnt.font.getsize(initial_text)
    w, h = draw.textsize(initial_text, font=fnt)

    # GENERATE FOLDER AND FILES
    style_folder = '_'.join(map(str, [
#        W,
#        H,
#        font_size,
#        text_color.replace('#',''),
#        bg_color.replace('#',''),
        font_face_for_filepath
    ]))
    if not os.path.isdir(style_folder):
        os.mkdir(style_folder)

    frames = []
    for t in range(seconds, -1, -1):
        text = str(datetime.timedelta(seconds=t))[slice_position:]
        img = Image.new('RGB', (W,H), color=bg_color)
        draw = ImageDraw.Draw(img)
        if auto_center:
            if auto_center_each_frame:
                (width, baseline), (offset_x, offset_y) = fnt.font.getsize(text)
                w, h = draw.textsize(text, font=fnt)
            x = (W-w-offset_x)/2
            y = (H-h-offset_y)/2
        draw.text((x,y), text, fill=text_color, font=fnt)
        frames.append(img)

    if make_gif:
        gif_filename = os.path.join(style_folder, str(datetime.timedelta(seconds=seconds)).replace(':', '_') + '.gif')
        frames[0].save(
            gif_filename,
            format='GIF',
            append_images=frames[1:],
            save_all=True,
            duration=int(1000/fps),
            loop=0,
            transparency=0,
            disposal=2
        )

    if make_zero:
        zero_filename = 'zero.png'
        frames[len(frames) - 1].save(os.path.join(style_folder, zero_filename))
        
    if make_static:
        first_frame_filename = str(datetime.timedelta(seconds=seconds)).replace(':', '_') + '-static.png'
        frames[0].save(os.path.join(style_folder, first_frame_filename))

    if make_mp4:
        video_filename = os.path.join(style_folder, str(datetime.timedelta(seconds=seconds)).replace(':', '_') + '.mp4')
        import numpy as np
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
        video = cv2.VideoWriter(video_filename,fourcc, fps, frames[0].size)
        for frame in frames:
            imtemp = frame.copy()
            video.write(np.array(imtemp))
        video.release()

    for frame in frames:
        frame.close()

In [4]:
times = [
'1:00',
'2:00',
'3:00',
'5:00',
'10:00',
'15:00',
'20:00',
'25:00',
'30:00',
'50:00'
]

In [5]:
times = [
'1:00',
'2:00'
]

In [6]:
fonts_and_sizes = [
    ['/Library/Fonts/Arial Rounded Bold.ttf', 590, 200],
    ['/Users/paulo/Library/Fonts/Market_Deco.ttf', 475, 215],
    ['/Users/paulo/Library/Fonts/orange juice 2.0.ttf', 510, 215],
    ['/Users/paulo/Library/Fonts/gigfont.ttf', 635, 205],
    ['/Library/Fonts/Arial.ttf', 550, 210],
    ['/Library/Fonts/Arial Bold.ttf', 560, 215],
    ['/Library/Fonts/Arial Black.ttf', 660, 215],
    ['/Users/paulo/Library/Fonts/From Cartoon Blocks.ttf', 660, 200],
    ['/Library/Fonts/Bodoni 72.ttc', 530, 210],
    ['/Library/Fonts/Chalkboard.ttc', 520, 205],
    ['/System/Library/Fonts/Courier.dfont', 645, 190],
    ['/Library/Fonts/DIN Alternate Bold.ttf', 475, 210]
]

In [8]:
print ('Generating assets:\n')
for f in fonts_and_sizes:
    config['font_face'] = f[0]
    config['W'] = int(f[1]) # 660 
    config['H'] = int(f[2]) # 215
    config['auto_size'] = not (config['W'] and config['H'])
    
   
    for t in times:
        
        config['make_static'] = False
        config['make_gif'] = False
        config['make_mp4'] = True
        config['make_zero'] = False
        config['text_color'] = '#000000'

        t0 = int(round(time.time() * 1000))
        config['time'] = t
        generate_countdown(config)

        config['make_static'] = True
        config['make_gif'] = False
        config['make_mp4'] = False
        config['make_zero'] = False
        config['text_color'] = '#000000'
        generate_countdown(config)

        t1 = int(round(time.time() * 1000))
        print (t.rjust(8) + ' ' + str(t1-t0).rjust(8) + 'ms    ' + config['font_face'])
                
    config['make_static'] = False
    config['make_gif'] = False
    config['make_mp4'] = False
    config['make_zero'] = True
    config['text_color'] = '#BA2121'
    generate_countdown(config)

Generating assets:

    1:00      459ms    /Library/Fonts/Arial Rounded Bold.ttf
    2:00      787ms    /Library/Fonts/Arial Rounded Bold.ttf
    1:00      308ms    /Users/paulo/Library/Fonts/Market_Deco.ttf
    2:00      819ms    /Users/paulo/Library/Fonts/Market_Deco.ttf
    1:00     2890ms    /Users/paulo/Library/Fonts/orange juice 2.0.ttf
    2:00     5229ms    /Users/paulo/Library/Fonts/orange juice 2.0.ttf
    1:00      549ms    /Users/paulo/Library/Fonts/gigfont.ttf
    2:00     1148ms    /Users/paulo/Library/Fonts/gigfont.ttf
    1:00      395ms    /Library/Fonts/Arial.ttf
    2:00      786ms    /Library/Fonts/Arial.ttf
    1:00      400ms    /Library/Fonts/Arial Bold.ttf
    2:00      859ms    /Library/Fonts/Arial Bold.ttf
    1:00      550ms    /Library/Fonts/Arial Black.ttf
    2:00     1079ms    /Library/Fonts/Arial Black.ttf
    1:00     3137ms    /Users/paulo/Library/Fonts/From Cartoon Blocks.ttf
    2:00     5193ms    /Users/paulo/Library/Fonts/From Cartoon Blocks.ttf
  